In [18]:
### param
file_name = 'data/EVT_2023_05_10_13_20_31_F_gps.txt'

In [19]:
with open(file_name, encoding='utf-8') as f:
    texts = f.read().split('\n')

In [20]:
import numpy as np
import pandas as pd

###
columns_name = ["index","time","text","info"]
###

raw_data = []
data = []
for t in texts:
    if(t == ""):
        data.append(raw_data)
        raw_data = []
    else:
        raw_data.append(t)

data = pd.DataFrame(data,columns=columns_name)
data = data.set_index("index")
data

,time,text,info
index,,,
1,"00:00:00,000 --> 00:00:00,100","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
2,"00:00:00,100 --> 00:00:00,200","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
3,"00:00:00,200 --> 00:00:00,300","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
4,"00:00:00,300 --> 00:00:00,400","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
5,"00:00:00,400 --> 00:00:00,500","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
...,...,...,...
197,"00:00:19,600 --> 00:00:19,700","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
198,"00:00:19,700 --> 00:00:19,800","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
199,"00:00:19,800 --> 00:00:19,900","<font size=""0"" color=""#000000"">{\an7}gsensori,...",";CAR,0,0,0,0.0,0,0,0,0,0,0,0,0</font>"


In [21]:
clean_data = data.drop(["info"],axis=1)

acceleration_index,acceleration_x,acceleration_y,acceleration_z = [],[],[],[]
# 加速度
for i in clean_data.index:
    text = str(clean_data.text[i])
    index = text.find("gsensori")
    acceleration = text[index+9+6:index+9+6+11].split(",")
    if(len(acceleration) != 3):
        continue
    acceleration_index.append(i)
    acceleration_x.append(acceleration[0])
    acceleration_y.append(acceleration[1])
    acceleration_z.append(acceleration[2])

acceleration = pd.DataFrame([acceleration_index,acceleration_x,acceleration_y,acceleration_z]).T
acceleration = acceleration.rename(columns={0:"index",1:"acceleration_x",2:"acceleration_y",3:"acceleration_z"})
acceleration = acceleration.set_index("index")

# 經緯度
#! 文字数で指定しているので緯度経度が大きく変化するとエラーが出る
pos_index,latitude,latitude_str,longitude,longitude_str = [],[],[],[],[]
for i in clean_data.index:
    text = str(clean_data.text[i])
    index = text.find("GPRMC")
    if(index == -1):
        continue
    pos_index.append(i)
    latitude.append(text[index+5+13:index+5+13+11])
    latitude_str.append(text[index+5+13+11+1:index+5+13+11+1+1])
    longitude.append(text[index+5+13+11+2+1:index+5+13+11+1+2+12])
    longitude_str.append(text[index+5+13+11+2+1+12+1:index+5+13+11+1+2+12+1+1])

latitude = [float(e) for e in latitude]
latitude_str = [str(e) for e in latitude_str]
longitude = [float(w) for w in longitude]
longitude_str = [str(w) for w in longitude_str]

position = pd.DataFrame([pos_index,latitude,latitude_str,longitude,longitude_str]).T
position = position.rename(columns={0:"index",1:"latitude",2:"latitude_str",3:"longitude",4:"longitude_str"})
position = position.set_index("index")

In [22]:
position["latitude"] = position["latitude"].astype(float)
position["latitude"] = position["latitude"]/100
position["longitude"] = position["longitude"].astype(float)
position["longitude"] = position["longitude"]/100

In [30]:
pd.Series([file_name for i in range(len(df))]).astype(str)

0      data/EVT_2023_05_10_13_20_31_F_gps.txt
1      data/EVT_2023_05_10_13_20_31_F_gps.txt
2      data/EVT_2023_05_10_13_20_31_F_gps.txt
3      data/EVT_2023_05_10_13_20_31_F_gps.txt
4      data/EVT_2023_05_10_13_20_31_F_gps.txt
                        ...                  
195    data/EVT_2023_05_10_13_20_31_F_gps.txt
196    data/EVT_2023_05_10_13_20_31_F_gps.txt
197    data/EVT_2023_05_10_13_20_31_F_gps.txt
198    data/EVT_2023_05_10_13_20_31_F_gps.txt
199    data/EVT_2023_05_10_13_20_31_F_gps.txt
Length: 200, dtype: object

In [33]:
df = pd.concat([acceleration,position],axis=1)
df = df.assign(file_name=file_name)
df

,acceleration_x,acceleration_y,acceleration_z,latitude,latitude_str,longitude,longitude_str,file_name
index,,,,,,,,
1,-56,001,120,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
2,-58,-06,109,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
3,-57,-06,104,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
4,-52,-02,115,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
5,-53,-07,108,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
...,...,...,...,...,...,...,...,...
196,-47,-04,110,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
197,-46,-03,112,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt
198,-50,-02,107,NaN,NaN,NaN,NaN,data/EVT_2023_05_10_13_20_31_F_gps.txt


In [24]:
import folium
folium_map = folium.Map(location=[position.latitude.mean(),position.longitude.mean()], zoom_start=15)

for i in df.dropna().index:
    folium.Marker(
        location=[position.latitude[i],position.longitude[i]],
        popup="",
        icon=folium.Icon(color="blue",icon="info-sign")
    ).add_to(folium_map)

folium_map